In [2]:
#cross validation between decision tree, extra tree, random forest, g boost, xg boost, svc
#add training data (cover type 1,2)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import*
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
standard_scaler = StandardScaler()

train = pd.read_csv('./train.csv').iloc[:, 1:]
def merge(df, count):
    if count >= 5: ###cover type 1,2를 얼마만큼 중복생성할 것인지 결정.### 
        return df
    add_1 = train.loc[train['Cover_Type'] == 1]
    add_2 = train.loc[train['Cover_Type'] == 2]
    new_df = pd.concat([add_1, add_2, df], axis=0)
    return merge(new_df, count+1)

dfX = train.iloc[:, :-1]
dfy = train.iloc[:, [-1]]


num = 5 #교차검증 횟수
#결과를 저장할 리스트
re_tree = []
re_extra = []
re_xg = []
re_svc = []
re_gb = []
re_random = []

#hyper parameters for decision tree, random forest, xg boost
n_es = 200
m_d = 10

for i in range(num):
    X_train, X_test, y_train, y_test = train_test_split(dfX, dfy, test_size=0.3)
    #train, test 데이터가 분리된 상태에서 train 데이터에 대해서만 covertype 1,2 추가 
    new_train = pd.concat([X_train, y_train], axis=1)
    new_train = merge(new_train, 0)
    X_train = new_train.iloc[:, :-1]
    y_train = new_train.iloc[:, [-1]]
    simple_tree = DecisionTreeClassifier(criterion = 'entropy', max_depth=m_d).fit(X_train, y_train)
    extra = ExtraTreesClassifier(max_depth=m_d, n_estimators=n_es).fit(X_train, y_train.values.ravel())
    random = RandomForestClassifier(n_estimators=n_es, max_depth=m_d).fit(X_train, y_train.values.ravel())
    xg = XGBClassifier(n_estimators=n_es, learning_rate=0.1, max_depth=m_d, objective='multi:softmax') 
    g_boost = GradientBoostingClassifier(n_estimators=200, max_depth=10).fit(X_train, y_train.values.ravel())
    result = xg.fit(X_train, y_train.values.ravel())
    model_svc = make_pipeline(StandardScaler(), SVC(C=1000, gamma = 0.1)).fit(X_train, y_train.values.ravel())
    pred_y1 = simple_tree.predict(X_test)
    pred_y2 = extra.predict(X_test)
    pred_y3 = result.predict(X_test)
    pred_y4 = model_svc.predict(X_test)
    pred_y5 = random.predict(X_test)
    pred_y6 = g_boost.predict(X_test)
    #print(tree1.feature_importances_)
    
    re_tree.append(accuracy_score(y_test, pred_y1))
    #print(classification_report(y_test, pred_y1, target_names = ['1', '2', '3', '4', '5', '6', '7']))
    re_extra.append(accuracy_score(y_test, pred_y2))
    #print(classification_report(y_test, pred_y2, target_names = ['1', '2', '3', '4', '5', '6', '7']))
    re_xg.append(accuracy_score(y_test, pred_y3))
    #print(classification_report(y_test, pred_y3, target_names = ['1', '2', '3', '4', '5', '6', '7']))
    re_svc.append(accuracy_score(y_test, pred_y4))
    re_random.append(accuracy_score(y_test, pred_y5))
    re_gb.append(accuracy_score(y_test, pred_y6))
    #print(classification_report(y_test, pred_y4, target_names = ['1', '2', '3', '4', '5', '6', '7']))

print('[simple decision tree]')
print(np.mean(re_tree), np.std(re_tree))
print('[ExtraTrees]')
print(np.mean(re_extra), np.std(re_random))
print('[Random Forest]')
print(np.mean(re_random), np.std(re_random))
print('[Gradiant boost]')
print(np.mean(re_gb), np.std(re_random))
print('[xgboost]')
print(np.mean(re_xg), np.std(re_xg))
print('[SVC]')
print(np.mean(re_svc), np.std(re_svc))

[simple decision tree]
0.738315696649 0.0
[ExtraTrees]
0.570987654321 0.0
[Random Forest]
0.645723104056 0.0
[Gradiant boost]
0.925264550265 0.0
[xgboost]
0.924382716049 0.0
[SVC]
0.898368606702 0.0


In [ ]:
# submit based validation
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
train = pd.read_csv('./train.csv').iloc[:, 1:]
def merge(df, count):
    if count >= 174: ###cover type 1,2를 얼마만큼 중복생성할 것인지 결정.### 
        return df
    add_1 = train.loc[train['Cover_Type'] == 1]
    add_2 = train.loc[train['Cover_Type'] == 2]
    new_df = pd.concat([add_1, add_2, df], axis=0)
    return merge(new_df, count+1)

new_train = merge(train, 0)

dfX = new_train.iloc[:, :-1]
dfy = new_train.iloc[:, [-1]]
print(len(dfy))

###n_estimators, max_depth 수정### 
model = XGBClassifier(n_estimators=200, learning_rate=0.1, max_depth=10, objective='multi:softmax')  
#model = XGBClassifier(n_estimators=400, max_depth=7, min_child_weight=4, objective='multi:softmax', nthread = -1)
#model = GradientBoostingClassifier(n_estimators=200, max_depth=10) #너무 오래걸림... 약간 손해보더라도 xgboost로 수행
result = model.fit(dfX, dfy.values.ravel())

#predict
test = pd.read_csv('./test.csv').iloc[:, 1:]
test_y = result.predict(test)
test_y = pd.DataFrame(test_y)

#submit file 생성
ids = pd.read_csv('./test.csv').iloc[:, :1]
submit = pd.concat([ids, test_y], axis=1)
submit.columns = ['Id', 'Cover_Type']
submit_group = submit.groupby('Cover_Type')
print(submit_group.count())
submit.to_csv("./Submission.csv", ',', index=False)